<a href="https://colab.research.google.com/github/Ishan075/Phishing-Detection-using-AI/blob/main/Phishing_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install imbalanced-learn
!pip uninstall whois -y
!pip install python-whois
!pip install whois-parser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.1 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.models import Model
from sklearn.ensemble import VotingClassifier
import re
from urllib.parse import urlparse
import whois
import datetime
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup
import signal

In [ ]:
#import re
import whois
import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
from whois_parser import WhoisParser

def having_ip(url):
    try:
        ip = urlparse(url).netloc
        if re.match(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b', ip):
            return 1
        else:
            return 0
    except Exception as e:
        print(f"Error in having_ip: {e}")
        return 0

def have_at_sign(url):
    return 1 if '@' in url else 0

def get_length(url):
    return 1 if len(url) > 54 else 0

def get_depth(url):
    try:
        s = urlparse(url).path.split('/')
        return len([i for i in s if len(i) > 0])
    except Exception as e:
        print(f"Error in get_depth: {e}")
        return 0

def redirection(url):
    try:
        return 1 if '//' in urlparse(url).path else 0
    except Exception as e:
        print(f"Error in redirection: {e}")
        return 0

def https_token(url):
    try:
        return 1 if 'https' in urlparse(url).netloc else 0
    except Exception as e:
        print(f"Error in https_token: {e}")
        return 0

def tiny_url(url):
    try:
        shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs"
        return 1 if re.search(shortening_services, url) else 0
    except Exception as e:
        print(f"Error in tiny_url: {e}")
        return 0

def prefix_suffix(url):
    try:
        return 1 if '-' in urlparse(url).netloc else 0
    except Exception as e:
        print(f"Error in prefix_suffix: {e}")
        return 0


def domain_age(domain_name):
    try:
        whois_info = whois.whois(domain_name)
        creation_date = whois_info.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        age = (datetime.datetime.now() - creation_date).days
        return 0 if age > 180 else 1
    except Exception as e:
        print(f"Error in domain_age: {e}")
        return 1  # Default value

def web_traffic(url):
    try:
        domain = urlparse(url).netloc
        if not domain:
            print(f"Invalid domain for URL: {url}")
            return 1  # Default value
        alexa_rank_url = f"https://www.alexa.com/siteinfo/{domain}"
        response = requests.get(alexa_rank_url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        rank = soup.find('div', {'class': 'rankmini-rank'})
        if rank:
            rank_value = int(rank.get_text().strip().replace(',', ''))
            return 0 if rank_value < 100000 else 1
        else:
            print(f"No Alexa rank found for {domain}")
            return 1  # Default value
    except Exception as e:
        print(f"Error in web_traffic: {e}")
        return 1  # Default value

In [ ]:
import whois

try:
    result = whois.whois("google.com")
    print(result)
except Exception as e:
    print(f"WHOIS test failed: {e}")

{
  "domain_name": "GOOGLE.COM",
  "registrar": "MarkMonitor, Inc.",
  "registrar_url": "http://www.markmonitor.com",
  "reseller": null,
  "whois_server": "whois.markmonitor.com",
  "referral_url": null,
  "updated_date": [
    "2019-09-09 15:39:04",
    "2024-08-02 02:17:33+00:00"
  ],
  "creation_date": [
    "1997-09-15 04:00:00",
    "1997-09-15 07:00:00+00:00"
  ],
  "expiration_date": [
    "2028-09-14 04:00:00",
    "2028-09-13 07:00:00+00:00"
  ],
  "name_servers": [
    "NS1.GOOGLE.COM",
    "NS2.GOOGLE.COM",
    "NS3.GOOGLE.COM",
    "NS4.GOOGLE.COM"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited",
    "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited",
    "serverTransferProhibited https://icann.org/epp#serverTransferProhibited",
    "serverUpdateProhibited ht

In [ ]:
#Trying and testing for the domain_age and web traffic
print(domain_age("google.com"))  # Expected: 0
print(domain_age("fakephishingsite123.com"))  # Expected: 1 or error message


0
Error in domain_age: No match for "FAKEPHISHINGSITE123.COM".
>>> Last update of whois database: 2025-01-21T10:22:10Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsoring
registrar.  Users may consult the sponsoring registrar's Whois database to
view the registrar's reported date of expiration for this registration.

TERMS OF USE: You are not authorized to access or query our Whois
database through the use of electronic processes that are high-volume and
automated except as reasonably necessary to register domain names or
modify existing registrations; the Data in VeriSign Global Registry
Services' ("VeriSign") Whois database is provided by VeriSign for
information purposes only, and to assist persons in obtaining information
about or related to a

In [ ]:
#Trying and testing for the web traffic
print(web_traffic("https://www.google.com"))  # Expected: 0
print(web_traffic("https://www.fakephishingsite123.com"))  # Expected: 1 or "No Alexa rank found"

No Alexa rank found for www.google.com
1
No Alexa rank found for www.fakephishingsite123.com
1


In [ ]:
# Step 2: Load Dataset
# Load a CSV dataset that contains URLs and their corresponding labels (0 for legitimate, 1 for phishing).
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(next(iter(uploaded)))  # Replace with your dataset file

# Check for the actual column name for labels
print(data.columns) # Print the column names to see the actual label column name

# Assuming the label column is named 'Label' (case-sensitive)
# Convert labels to binary (0 for 'good', 1 for 'bad')
data['label'] = data['Label'].apply(lambda x: 1 if x == 'bad' else 0)
# Replace 'Label' with the actual name if different

# Alternatively, if the column name has spaces or special characters, use:
# data['label'] = data[data.columns[1]].apply(lambda x: 1 if x == 'bad' else 0) # Assuming label is the second column

Saving phishing_site_urls (2).csv to phishing_site_urls (2) (1).csv
Index(['URL', 'Label'], dtype='object')


In [ ]:
import pandas as pd
import signal

# Define a TimeoutException class
class TimeoutException(Exception):
    pass

# Define a timeout handler
def timeout_handler(signum, frame):
    raise TimeoutException

# Attach the timeout handler to the signal module
signal.signal(signal.SIGALRM, timeout_handler)

# Feature extraction function
def extract_features(url):
    return [
        having_ip(url),
        have_at_sign(url),
        get_length(url),
        get_depth(url),
        redirection(url),
        https_token(url),
        tiny_url(url),
        prefix_suffix(url),
        domain_age(urlparse(url).netloc),
       web_traffic(url)
    ]

# Initialize a list to store features
features = []

# Process each URL in the dataset
for i, url in enumerate(data['URL']):
    try:
        signal.alarm(45)  # Set a timeout of 45 seconds
        features.append(extract_features(url))
        signal.alarm(0)  # Reset the alarm
        print(f"Processed URL {i + 1}/{len(data)}: {url}")
    except TimeoutException:
        features.append([1] * 10)  # Default feature values for timed-out URLs
        print(f"Timeout occurred for URL {i + 1}: {url}")
    except Exception as e:
        features.append([1] * 10)  # Default feature values for other errors
        print(f"Error processing URL {i + 1}: {url} - {e}")

# Convert the list of features to a DataFrame
features_df = pd.DataFrame(features)

# Check the feature DataFrame for validation
print(features_df.describe())

Error in domain_age: module 'whois' has no attribute 'whois'
Error in web_traffic: 'NoneType' object has no attribute 'get_text'
Processed URL 1/549346: nobell.it/70ffb52d079109dca5664cce6f317373782/login.SkyPe.com/en/cgi-bin/verification/login/70ffb52d079109dca5664cce6f317373/index.php?cmd=_profile-ach&outdated_page_tmpl=p/gen/failed-to-load&nav=0.5.1&login_access=1322408526
Error in domain_age: module 'whois' has no attribute 'whois'
Error in web_traffic: 'NoneType' object has no attribute 'get_text'
Processed URL 2/549346: www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php
Error in domain_age: module 'whois' has no attribute 'whois'
Error in web_traffic: 'NoneType' object has no attribute 'get_text'
Processed URL 3/549346: serviciosbys.com/paypal.cgi.bin.get-into.herf.secure.dispatch35463256rzr321654641dsf654321874/href/href/href/secure/center/update/limit/seccure/4d7a1ff5c55825a2e632a679c2fd5353/
Error in domain_age: module 'whois' has no attribute 'w

KeyboardInterrupt: 

In [ ]:
# Step 4: Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features_df, labels)

In [ ]:
# Step 5: Train-Test Split
# Split the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
# Step 6: Feature Scaling
# Standardize features by removing the mean and scaling to unit variance.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 7: Build Individual Models
# Define different models for the ensemble

# 7.1: Build CNN-Bidirectional LSTM Model
input_layer = Input(shape=(X_train_scaled.shape[1], 1))
cnn_layer = Conv1D(filters=256, kernel_size=3, activation='relu')(input_layer)
batch_norm = BatchNormalization()(cnn_layer)
global_pool = GlobalMaxPooling1D()(batch_norm)
bilstm_layer = Bidirectional(LSTM(100, return_sequences=True))(tf.expand_dims(global_pool, axis=-1))
bilstm_layer_2 = Bidirectional(LSTM(50))(bilstm_layer)
dropout_layer = Dropout(0.5)(bilstm_layer_2)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

nn_model = Model(inputs=input_layer, outputs=output_layer)
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Train Neural Network Model
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
nn_model.fit(X_train_scaled[..., np.newaxis], y_train, epochs=20, batch_size=64, validation_data=(X_test_scaled[..., np.newaxis], y_test), class_weight=class_weights)

# 7.2: Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train_scaled, y_train)

# 7.3: Logistic Regression
lr_model = LogisticRegression(random_state=42, class_weight='balanced')
lr_model.fit(X_train_scaled, y_train)


In [ ]:
# Step 8: Create an Ensemble Model
# Use VotingClassifier to combine the models for better predictions.
ensemble_model = VotingClassifier(estimators=[
    ('nn', nn_model),
    ('rf', rf_model),
    ('lr', lr_model)
], voting='soft')
# Train the Ensemble Model
ensemble_model.fit(X_train_scaled, y_train)

In [ ]:
# Step 9: Evaluate the Ensemble Model
# Use the testing set to evaluate the performance of the ensemble model.
y_pred = ensemble_model.predict(X_test_scaled)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

In [ ]:
# Step 10: Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
